# Adaptive Sampling

In this tutorial you will learn:
    
    ...
    
    

In [ ]:
from bluesky import RunEngine
from bluesky.plans import count
from simulated_hardware import detector, load_sample, unload_sample, light
from utils.visualization import SummedImages, JupyterFigure
from bluesky_widgets.utils.streaming import stream_documents_into_runs
from bluesky_widgets.models.plot_specs import Axes, Figure

In [ ]:
axes_list = [Axes() for _ in range(9)]
figure_model = Figure(axes_list, title="Summed Exposures")
models = [SummedImages("detector_image", axes=axes) for axes in axes_list]
figure = JupyterFigure(figure_model)
figure

In [ ]:
RE = RunEngine()

In [ ]:
RE(load_sample(1))

In [ ]:
# Turn on light
RE(light(True))

In [ ]:
RE(count([detector], 10, delay=1), stream_documents_into_runs(models[0].add_run))